Build dataset


In [1]:
from typing import TypedDict, Type, Any, Callable

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from spark.config import views
from spark.create_session import create_session

from IPython.display import display

from fitter import Fitter, get_common_distributions

In [2]:
VIEWS = views("v3")
spark = create_session()

for view, file in VIEWS.items():
    df = spark.read.json(file)
    df.createOrReplaceTempView(view)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/27 16:02:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/27 16:02:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
sessions_full = spark.sql(f"SELECT * FROM sessions").toPandas()
sessions = spark.sql(f"SELECT DISTINCT user_id, track_id FROM sessions WHERE event_type like 'play' order by user_id, track_id").toPandas()
tracks = spark.sql(
    f"SELECT DISTINCT id, acousticness, danceability, duration_ms, energy, instrumentalness, key, liveness, loudness, popularity, EXTRACT(year from `release_date`) as release_year, speechiness, tempo, valence FROM tracks ").toPandas()

In [4]:
display(sessions_full)
display(sessions)
display(tracks)

,event_type,session_id,timestamp,track_id,user_id
0,play,124,2023-09-20T07:18:17,4flBNxpPLGVAdufOzRBIen,101
1,like,124,2023-09-20T07:18:30.069000,4flBNxpPLGVAdufOzRBIen,101
2,play,125,2023-04-27T00:20:57.181000,3uMYq07Kj5m564OQwdSCrD,101
3,play,126,2023-02-11T05:34:54.160000,2RChe0r2cMoyOvuKobZy44,101
4,advertisment,126,2023-02-11T05:34:56.685000,None,101
...,...,...,...,...,...
580241,like,102015,2023-11-12T09:06:01.022000,4cLdpErILMO8Db8pQVAVcZ,1100
580242,play,102015,2023-11-12T09:09:17.564000,3j8ja2Hq824OaRqIENJPTH,1100
580243,like,102015,2023-11-12T09:12:19.739000,3j8ja2Hq824OaRqIENJPTH,1100
580244,skip,102015,2023-11-12T09:12:58.093000,3j8ja2Hq824OaRqIENJPTH,1100


,user_id,track_id
0,101,03LNdMgu3l3Ldc3QMl1bvZ
1,101,08QXSGwwI8Qu4IQEV3AfM6
2,101,09WxJCWFMWAxTHBLLelpDS
3,101,0BVCEJJFVsb8nrQGI11Dj2
4,101,0ChHBmij3WHKGmyan2tPST
...,...,...
318740,1100,7qRmdDl9dnHkFT4MMboG7Y
318741,1100,7rPbzHewbn20oGdUuIxhCa
318742,1100,7rvILFySMopzPchZq9Bnmf
318743,1100,7s41ZGjQB5Ur8T0fQlk5uM


,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,popularity,release_year,speechiness,tempo,valence
0,3KfbEIOC7YIv90FIfNSZpo,0.4490,0.688,146333,0.435,0.000000,9,0.1130,-11.359,74,1965,0.0323,103.239,0.435
1,1dxbAIfCASqv6jix2R1Taj,0.7840,0.698,148413,0.293,0.010600,0,0.0936,-11.361,55,1967,0.0332,117.613,0.724
2,17PXXzOygMyXXUNLngVN5u,0.8820,0.539,173733,0.114,0.009020,2,0.1060,-20.575,60,1970,0.0346,141.378,0.608
3,3Um9toULmYFGCpvaIPFw7l,0.4470,0.283,233000,0.716,0.000000,1,0.3990,-9.632,72,1971,0.0986,201.960,0.828
4,0GjEhVFGZW8afUYGChu3Rr,0.3580,0.543,230400,0.870,0.000939,9,0.7920,-6.514,81,1976,0.0428,100.804,0.754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22407,3VbsXTdxBl6IavGjFZbl3R,0.0394,0.720,225012,0.824,0.000000,10,0.0509,-5.086,53,2020,0.1280,165.998,0.333
22408,2zurBuX8TGOHkZLhye9Xme,0.0551,0.769,179080,0.922,0.000000,9,0.1940,-2.779,51,2020,0.1020,119.991,0.804
22409,3ev1ilsHdVbpXeT7FG6Kw7,0.6410,0.838,229187,0.461,0.000000,1,0.0825,-9.610,54,2017,0.2170,104.979,0.497
22410,5qTwnr6WbWeeUDS9dcl5g6,0.3370,0.444,303097,0.523,0.000000,1,0.1130,-4.522,54,2019,0.0286,185.872,0.420


In [5]:
# d = spark.sql(
#     """
#     SELECT s.user_id, s.track_id, s.weight, acousticness, danceability, duration_ms, energy, instrumentalness, key, liveness, loudness, popularity, EXTRACT(year from `release_date`) as release_year, speechiness, tempo, valence
#     FROM (
#         select user_id, track_id, sum(event_weight) as weight
#         from (
#             SELECT user_id, track_id, CASE WHEN event_type like 'like' THEN 1 ELSE 0.02 END as event_weight
#             FROM sessions
#             WHERE event_type like 'like' or event_type like 'play'
#             ) 
#         group by user_id, track_id
#     ) s
#     inner join tracks t on s.track_id = t.id
#     order by s.user_id, t.id
#     """).toPandas()
d = spark.sql(
    """
    SELECT s.user_id, s.track_id, s.weight, acousticness, danceability, duration_ms, energy, instrumentalness, key, liveness, loudness, popularity, EXTRACT(year from `release_date`) as release_year, speechiness, tempo, valence
    FROM (
        select user_id, track_id, sum(event_weight) as weight
        from (
            SELECT user_id, track_id, 1 as event_weight
            FROM sessions
            WHERE event_type like 'play'
            ) 
        group by user_id, track_id
    ) s
    inner join tracks t on s.track_id = t.id
    order by s.user_id, t.id
    """).toPandas()
d

,user_id,track_id,weight,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,popularity,release_year,speechiness,tempo,valence
0,101,03LNdMgu3l3Ldc3QMl1bvZ,1,0.545000,0.674,182549,0.637,0.000002,0,0.0483,-3.400,52,2014,0.0330,139.867,0.755
1,101,08QXSGwwI8Qu4IQEV3AfM6,1,0.552000,0.697,162960,0.633,0.000000,7,0.1280,-6.983,60,2019,0.0793,139.074,0.716
2,101,09WxJCWFMWAxTHBLLelpDS,1,0.127000,0.842,271973,0.831,0.000000,1,0.0881,-7.072,60,1996,0.2420,92.053,0.814
3,101,0BVCEJJFVsb8nrQGI11Dj2,1,0.005220,0.656,235253,0.923,0.000000,2,0.3260,-3.541,56,2011,0.0339,139.930,0.652
4,101,0ChHBmij3WHKGmyan2tPST,1,0.128000,0.829,208667,0.733,0.194000,8,0.0780,-8.590,57,1999,0.0342,102.128,0.685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318740,1100,7qRmdDl9dnHkFT4MMboG7Y,1,0.000333,0.505,173827,0.906,0.000127,7,0.2900,-5.127,56,2014,0.0399,107.855,0.553
318741,1100,7rPbzHewbn20oGdUuIxhCa,1,0.775000,0.651,264160,0.307,0.000000,11,0.0956,-9.304,57,2019,0.0327,140.083,0.157
318742,1100,7rvILFySMopzPchZq9Bnmf,1,0.121000,0.578,261307,0.940,0.000261,9,0.2580,-2.751,52,2016,0.1880,89.805,0.609
318743,1100,7s41ZGjQB5Ur8T0fQlk5uM,1,0.014600,0.447,205573,0.765,0.000000,6,0.0750,-5.001,66,2007,0.0433,169.853,0.365


In [6]:
from scipy import stats

users = d['user_id'].unique()
items = d['track_id'].unique()


_features = pd.DataFrame(items, columns=['track_id']).merge(tracks, left_on="track_id", right_on="id", how="inner", validate="1:1" )
features = stats.zscore(_features.drop(['id', 'track_id'], axis=1))


assert items.shape[0] == features.shape[0]
_features

,track_id,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,popularity,release_year,speechiness,tempo,valence
0,03LNdMgu3l3Ldc3QMl1bvZ,03LNdMgu3l3Ldc3QMl1bvZ,0.54500,0.674,182549,0.637,0.000002,0,0.0483,-3.400,52,2014,0.0330,139.867,0.755
1,08QXSGwwI8Qu4IQEV3AfM6,08QXSGwwI8Qu4IQEV3AfM6,0.55200,0.697,162960,0.633,0.000000,7,0.1280,-6.983,60,2019,0.0793,139.074,0.716
2,09WxJCWFMWAxTHBLLelpDS,09WxJCWFMWAxTHBLLelpDS,0.12700,0.842,271973,0.831,0.000000,1,0.0881,-7.072,60,1996,0.2420,92.053,0.814
3,0BVCEJJFVsb8nrQGI11Dj2,0BVCEJJFVsb8nrQGI11Dj2,0.00522,0.656,235253,0.923,0.000000,2,0.3260,-3.541,56,2011,0.0339,139.930,0.652
4,0ChHBmij3WHKGmyan2tPST,0ChHBmij3WHKGmyan2tPST,0.12800,0.829,208667,0.733,0.194000,8,0.0780,-8.590,57,1999,0.0342,102.128,0.685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22407,5leINazMrsgEq4SYHXWTaC,5leINazMrsgEq4SYHXWTaC,0.51600,0.617,165853,0.497,0.000002,5,0.0634,-5.430,54,2016,0.0630,169.580,0.919
22408,2GB8OypbvrvCee61FKx5dp,2GB8OypbvrvCee61FKx5dp,0.13900,0.497,137480,0.590,0.000007,2,0.3780,-11.203,52,1968,0.0296,142.027,0.969
22409,1Rr17slUHwVRkDz4A2TUcM,1Rr17slUHwVRkDz4A2TUcM,0.00144,0.532,248467,0.931,0.263000,11,0.1590,-3.662,52,2006,0.0467,113.424,0.383
22410,1gVnckXtTdCoLwtiPPqlQS,1gVnckXtTdCoLwtiPPqlQS,0.40000,0.592,316546,0.371,0.000367,9,0.0953,-9.965,64,2019,0.0437,127.966,0.197


In [7]:
from lightfm.data import Dataset

assert d['track_id'].unique().shape[0] == features.shape[0]

dataset = Dataset()
dataset.fit(
    users=users,
    items=items,
    item_features=features
)

num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 1000, num_items 22412.


In [8]:
(interactions, weights) = dataset.build_interactions(d[['user_id', 'track_id', 'weight']].apply(tuple, axis=1))

print(repr(interactions))

<1000x22412 sparse matrix of type '<class 'numpy.int32'>'
	with 318745 stored elements in COOrdinate format>


In [9]:
from lightfm.cross_validation import random_train_test_split

(train, test) = random_train_test_split(interactions)


In [10]:
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k, reciprocal_rank


In [11]:
feature_names = d.drop(['user_id', 'track_id', 'weight'], axis=1).columns

item_features = dataset.build_item_features(
    ((i, feature_names) for i in items), 
    normalize=False)
print(repr(item_features))

<22412x22425 sparse matrix of type '<class 'numpy.float32'>'
	with 313768 stored elements in Compressed Sparse Row format>


In [12]:
model_h = LightFM(
    loss='warp',
    learning_rate=0.0,
    item_alpha=1e-6,
    user_alpha=1e-6,
    no_components=30,
)
model_h.fit(
    interactions=train,
    item_features=item_features,
    epochs=5,
    num_threads=12,
    verbose=True)


Epoch: 100%|██████████| 5/5 [00:10<00:00,  2.04s/it]


In [13]:
train_auc_h = auc_score(model_h, train, 
                        item_features=item_features,
                        num_threads=12).mean()
test_auc_h = auc_score(model_h, test, 
                       train_interactions=train, 
                       item_features=item_features, 
                       num_threads=12).mean()

train_precision_h = precision_at_k(model_h, train, k=10,
                                   item_features=item_features, 
                                   num_threads=12).mean()
test_precision_h = precision_at_k(model_h, test, k=10, 
                                  train_interactions=train,
                                  item_features=item_features, 
                                  num_threads=12).mean()

train_recall_h = recall_at_k(model_h, train, k=10, 
                             item_features=item_features,
                             num_threads=12).mean()
test_recall_h = recall_at_k(model_h, test, k=10, 
                            train_interactions=train,
                            item_features=item_features, 
                            num_threads=12).mean()

train_reciprocal_rank_h = reciprocal_rank(model_h, train,
                                          item_features=item_features,
                                          num_threads=12).mean()
test_reciprocal_rank_h = reciprocal_rank(model_h, test, 
                                         train_interactions=train,
                                         item_features=item_features,
                                         num_threads=12).mean()

print('AUC: train %.6f, test %.6f.' % (train_auc_h, test_auc_h))
print('Precision: train %.6f, test %.6f.' % (train_precision_h, test_precision_h))
print('Recall: train %.6f, test %.6f.' % (train_recall_h, test_recall_h))
print('Reciprocal rank: train %.6f, test %.6f.' % (train_reciprocal_rank_h, test_reciprocal_rank_h))


AUC: train 0.553368, test 0.498829.
Precision: train 0.026000, test 0.003300.
Recall: train 0.000999, test 0.000447.
Reciprocal rank: train 0.098102, test 0.018936.


In [14]:
predicted_scores = model_h.predict(148, np.arange(interactions.shape[1]), 
                                   item_features=item_features, 
                                   num_threads=12)
predicted_scores

array([0.04049674, 0.0651309 , 0.04002773, ..., 0.05452988, 0.03218829,
       0.05478507], dtype=float32)

In [15]:
indices = np.argpartition(predicted_scores, -10)[-10:]
indices

array([ 3638,  9894,  1456,  7906,  5446,  7561,  3961, 17732,  2713,
        2459])

In [16]:
pd.DataFrame(items).loc[indices]

,0
3638,5B8N5rPOmTVVGpuBMK2Vby
9894,1ZdhOMWyFR8Iv9eylMGYg2
1456,4xUqqie4bBKufHtlMuZS3k
7906,0qqLzlmKDj55Zmck2rdT33
5446,6BnRwKaF2bRuA3bbrPjBJg
7561,0qHMhBZqYb99yhX9BHcIkV
3961,7jeqHTL5LN4YEyMhiqf9Ei
17732,43hiD0QEeUysuWqDzd8GwA
2713,0lJEYLAJU5KtthQroas2ou
2459,5aIZ2jtxZ4o4GHQXTybkIa


In [17]:
d[d.user_id == 249]

,user_id,track_id,weight,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,popularity,release_year,speechiness,tempo,valence
47584,249,07BuyVse8pYAWd9DXD7B2D,1,0.000831,0.536,235240,0.968,0.000030,6,0.0920,-2.373,65,2015,0.0891,86.036,0.570
47585,249,07qHCBBSRswIsfs0waYdjC,1,0.799000,0.583,169846,0.418,0.000000,5,0.1090,-10.898,58,2019,0.0451,129.915,0.357
47586,249,0TU7xhMfZbI1okpLyHptFV,1,0.043800,0.679,237853,0.797,0.000000,2,0.0748,-4.627,54,2019,0.3380,117.018,0.609
47587,249,0XfTh16YcKPzdxMPTeKTAq,1,0.367000,0.610,261453,0.531,0.000000,2,0.3510,-6.123,52,2008,0.0239,85.979,0.228
47588,249,0aMqNFBj9KtPTD3c3tByRT,1,0.000788,0.443,138048,0.878,0.560000,8,0.0623,-6.717,51,1989,0.0418,136.567,0.683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47704,249,7JiCaZ93B0hdj3XwFqwn4W,1,0.371000,0.538,218760,0.786,0.002580,7,0.0517,-7.690,52,1989,0.0336,172.019,0.917
47705,249,7K5dzhGda2vRTaAWYI3hrb,1,0.011400,0.632,215414,0.701,0.000000,7,0.1080,-5.921,72,2016,0.1460,162.958,0.497
47706,249,7MevrbPAGEaw1IgkaBmv82,1,0.078200,0.685,195587,0.909,0.000001,0,0.3690,-3.797,60,2009,0.0320,115.043,0.864
47707,249,7gpuC3rLKkI7PoJcEnSIO6,1,0.879000,0.489,242867,0.308,0.000000,0,0.0959,-5.488,52,2000,0.0297,128.855,0.290


In [18]:

model_h.get_item_representations()

(array([-0.01096737,  0.01443374, -0.01015265, ...,  0.00466694,
         0.00557848,  0.00528683], dtype=float32),
 array([[ 0.01250516, -0.00901319,  0.01302429, ..., -0.00375146,
         -0.01483755, -0.01323879],
        [ 0.01772139,  0.01270564,  0.00121316, ..., -0.00215665,
          0.0029933 , -0.00141379],
        [ 0.01358402,  0.00758789, -0.01152616, ...,  0.01166518,
          0.01691847,  0.00218847],
        ...,
        [-0.00218298,  0.01168382, -0.00836872, ..., -0.00771284,
          0.00231354,  0.00652879],
        [-0.00771839,  0.00471733, -0.01033946, ..., -0.00903918,
         -0.00280672, -0.00656529],
        [-0.00117605, -0.00695348, -0.01563769, ..., -0.00458429,
         -0.00715612,  0.00693074]], dtype=float32))